## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [196]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ci.transltr.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ci.transltr.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    else:
                        dictionary_2[child['actor']['agent']] = [] 
                else:
                    dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [3]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(600)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.transltr.io/?source=ARS&id=57f95ee7-4b1e-4cb1-9169-3a582fb8909b
Done
Done
ara-aragorn Done 257
Done
ara-arax Done 190
Done
ara-bte Done 199
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 73
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.transltr.io/?source=ARS&id=471d6632-51c0-4065-8303-4231bf8e58f7
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 336
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-ca

kp-molecular Done 224
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 67
Done
kp-chp Done 60
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.0_Initial_ARS_three-hop_query_results_MESH:D056487.json
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
https://arax.transltr.io/?source=ARS&id=564542e4-77a6-422a-8484-3ddd89b0a0e0
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 0
Done
ara-bte Done 2
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
/User

In [ ]:
result_status = retrieve_ars_results(kcresult)

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [187]:
dict_workflows

{'A.3_KCNMA1': '57f95ee7-4b1e-4cb1-9169-3a582fb8909b',
 'A.8_EGFR_simple': '471d6632-51c0-4065-8303-4231bf8e58f7',
 'A.0_RHOBTB2_direct': '1a3f8054-ee71-434b-823c-cdbf4c0e3ba9',
 'A.2_RHOBTB2_twohop': 'c76c42a5-2f70-49df-bd2a-349e8a4cc12e',
 'A.9_EGFR_advanced': '1914d7fc-f9f8-488c-9abe-20ace5a3e507',
 'A.1_RHOBTB2': 'ccd90194-9511-458b-aedf-dd66c83737f2',
 'B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': 'c65c64bb-5917-4934-904f-5f879991a544',
 'B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '27869ff3-b1fb-4535-996f-9988eb371a24',
 'B.4_one-hop-gene-biological-process-or-activity_trapi': '014f649d-05b0-4bac-9c7e-d84de5747ca4',
 'B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': '5ee4186c-b4d1-4033-880e-382ee2e72f27',
 'B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': 'b2645024-47ee-4670-9207-2a2de1da5756',
 'B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi': 'fb03e329-52ee-4766-b86e-3c3e86d59778',
 'B.3_DILI-one

In [197]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 57f95ee7-4b1e-4cb1-9169-3a582fb8909b
Done
Done
ara-aragorn Done 257
Done
ara-arax Done 190
Done
ara-bte Done 199
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 73
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
A.8_EGFR_simple 471d6632-51c0-4065-8303-4231bf8e58f7
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 336
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-robokop Done 83
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili Done 0
A.0_RHOBTB2_direct 1a3f8054-ee71-434b-823c-cdbf4c0e3

Done
Error
ara-aragorn Error 0
Error
ara-arax Error 0
Error
ara-bte Error 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 10
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.1_parkinsons-crohns f2b6c478-f266-4899-959f-a3b2ceab3d58
Done
Done
ara-aragorn Done 382
Done
ara-arax Done 182
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 22
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Error
kp-icees-dili Error 0
D.3_ssri-heart-disease-one-hop 294fbf09-a7d4-496c-8775-8d6c4

In [189]:
 workflow_result_messages

{'A.3_KCNMA1': [{'pk_id': 'https://arax.ci.transltr.io/?source=ARS&id=57f95ee7-4b1e-4cb1-9169-3a582fb8909b',
   'ara-aragorn': 'Results: 200',
   'ara-arax': 'Results: 200',
   'ara-bte': 'Results: 200',
   'ara-unsecret': 'Results: 200',
   'kp-genetics': 'No Results: 200',
   'kp-molecular': 'Results: 200',
   'ara-explanatory': 'No Results: 200',
   'ara-improving': 'Results: 200',
   'kp-cam': 'No Results: 200',
   'kp-textmining': 'No Results: 200',
   'ara-aragorn-exp': 'Error: 404',
   'kp-openpredict': 'No Results: 200',
   'kp-icees': 'No Results: 200',
   'ara-robokop': 'Results: 200',
   'kp-chp': 'Results: 200',
   'kp-cohd': 'ARS Error: 200',
   'kp-icees-dili': 'No Results: 200'},
  {'ara-aragorn': ['infores:aragorn-ranker-ara'],
   'ara-arax': ['infores:rtx-kg2', 'infores:dgidb', 'infores:arax'],
   'ara-bte': ['infores:translator-biothings-explorer',
    'infores:gtopdb',
    'infores:automat.robokop'],
   'ara-unsecret': ['infores:molepro-kp'],
   'kp-genetics': [],
  

In [190]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine-with-curie
D.5_tryptophan-kynurenine
D.6_metformin-ferritin


In [198]:
## Convert mesages to a dataframe
col = []
final_dict2 = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][1].items():
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict2[key].append(value)

    final_dict2 = dict(final_dict2)
    
df2 = pd.DataFrame(final_dict2).T
df2.rename(columns=dict(zip(df2.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine-with-curie
D.5_tryptophan-kynurenine
D.6_metformin-ferritin


ValueError: arrays must all be same length

In [194]:
workflow_result_messages[k][1]

{'ara-aragorn': ['infores:intact',
  'infores:automat.intact',
  'infores:aragorn'],
 'ara-arax': [],
 'ara-bte': [],
 'kp-genetics': [],
 'kp-molecular': [],
 'ara-explanatory': [],
 'ara-improving': ['infores:string',
  'infores:spoke',
  'infores:improving-agent'],
 'kp-cam': [],
 'kp-textmining': [],
 'ara-aragorn-exp': [],
 'kp-openpredict': [],
 'kp-icees': [],
 'ara-robokop': [],
 'kp-chp': [],
 'kp-cohd': [],
 'kp-icees-dili': []}

In [195]:
k

'D.6_metformin-ferritin'

In [184]:
workflow_result_messages['A.0_RHOBTB2_direct'][1]

{'ara-arax': [],
 'ara-aragorn': [],
 'ara-bte': [],
 'ara-unsecret': [],
 'kp-genetics': [],
 'kp-molecular': [],
 'ara-explanatory': [],
 'ara-improving': ['infores:lincs',
  'infores:spoke',
  'infores:improving-agent'],
 'kp-cam': [],
 'kp-textmining': [],
 'ara-aragorn-exp': [],
 'kp-icees': [],
 'kp-openpredict': [],
 'ara-robokop': [],
 'kp-chp': [],
 'kp-cohd': [],
 'kp-icees-dili': []}

In [83]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.transltr.io/?source=ARS&id=1a3f80...,https://arax.transltr.io/?source=ARS&id=ccd901...,https://arax.transltr.io/?source=ARS&id=c76c42...,https://arax.transltr.io/?source=ARS&id=57f95e...,https://arax.transltr.io/?source=ARS&id=471d66...,https://arax.transltr.io/?source=ARS&id=1914d7...,https://arax.transltr.io/?source=ARS&id=564542...,https://arax.transltr.io/?source=ARS&id=c65c64...,https://arax.transltr.io/?source=ARS&id=27869f...,https://arax.transltr.io/?source=ARS&id=3a03c5...,...,https://arax.transltr.io/?source=ARS&id=f1c549...,https://arax.transltr.io/?source=ARS&id=014f64...,https://arax.transltr.io/?source=ARS&id=798c35...,https://arax.transltr.io/?source=ARS&id=d70406...,https://arax.transltr.io/?source=ARS&id=f2b6c4...,https://arax.transltr.io/?source=ARS&id=f608a6...,https://arax.transltr.io/?source=ARS&id=294fbf...,https://arax.transltr.io/?source=ARS&id=4161a2...,https://arax.transltr.io/?source=ARS&id=4f0f39...,https://arax.transltr.io/?source=ARS&id=396846...
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,...,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,...,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,No Results: 200,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503
kp-molecular,No Results: 200,No Results: 200,Error: 501,Results: 200,No Results: 200,No Results: 200,Error: 501,Error: 501,Error: 501,Error: 501,...,Results: 200,No Results: 200,No Results: 200,Error: 501,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501,Error: 501
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,...,ARS Error: 200,No Results: 200,ARS Error: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
kp-c

In [85]:
df2

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-arax,[],"[infores:rtx-kg2, infores:semmeddb, [PMID:2976...","[infores:rtx-kg2, infores:dgidb, infores:arax]","[infores:rtx-kg2, infores:dgidb, infores:arax]","[infores:rtx-kg2, infores:semmeddb, [PMID:7696...","[infores:rtx-kg2, infores:semmeddb, [PMID:7696...",[],[],[],[],...,"[infores:rtx-kg2, infores:semmeddb, [PMID:2747...","[infores:rtx-kg2, infores:goa, [PMID:23871208]...",[],[],"[infores:rtx-kg2, infores:semmeddb, [PMID:1100...","[infores:rtx-kg2, infores:semmeddb, [PMID:1701...","[infores:rtx-kg2, infores:semmeddb, [PMID:1040...",[],[],[]
ara-aragorn,[],"[infores:lincs, infores:spoke, -6.963625168134...","[72, [], infores:aragorn-ranker-ara, 1]","[1, [], infores:aragorn-ranker-ara, 1]",[],[],[],[],[],[],...,"[4, [], infores:aragorn-ranker-ara, 1]",[],"[537, [], infores:aragorn-ranker-ara, 1]",[],"[261, [], infores:aragorn-ranker-ara, 1]","[16, [], infores:aragorn-ranker-ara, 1]","[21, [], infores:aragorn-ranker-ara, 1]",[],[],"[[IntAct_1_norm_edges.jsonl], MI:0399, infores..."
ara-bte,[],"[[infores:semmeddb], [SEMMED Gene API], [PMID:...","[infores:translator-biothings-explorer, [edges...","[infores:translator-biothings-explorer, -, Fal...",[],"[infores:translator-biothings-explorer, pIC50,...","[infores:translator-biothings-explorer, [Cord1...","[[], [Clinical Risk KP API], [], 0.914, Logist...",[],[],...,"[infores:translator-biothings-explorer, False,...",[],[],[],[],[],"[[infores:semmeddb], [SEMMED Chemical API], [P...",[],[],[]
ara-unsecret,[],[],[],"[95.05, PMID:29195078, https://clue.io/cmap, 1...",[],[],[],[],[],[],...,"[95.64, PMID:29195078, https://clue.io/cmap, 1...",[],[],[],[],[],[],[],[],"[False, ""(\""\""identifiers_org_registry:reactom..."
kp-genetics,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
kp-molecular,[],[],[],"[95.05, PMID:29195078, https://clue.io/cmap, 1...",[],[],[],[],[],[],...,"[95.64, PMID:29195078, https://clue.io/cmap, 1...",[],[],[],[],[],[],[],[],[]
ara-explanatory,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
ara-improving,"[infores:lincs, infores:spoke, infores:improvi...","[infores:lincs, infores:spoke, infores:improvi...","[pmid:25184630, infores:civic, infores:spoke, ...","[infores:lincs, infores:spoke, infores:improvi...",[],[],[],[],[],[],...,[],[],[],[],"[infores:chembl, infores:spoke, infores:improv...","[infores:diseases, infores:spoke, infores:impr...","[infores:drugcentral, infores:spoke, infores:i...","[infores:bindingdb, infores:spoke, infores:imp...","[infores:bindingdb, infores:spoke, infores:imp...","[infores:string, infores:spoke, infores:improv..."
kp-cam,[],[],[],[],[],[],[],[],[],[],...,[],"[[infores:cam-kp], [infores:go-cam]]",[],[],[],[],[],[],[],[]
kp-textmining,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [ ]:
#df_copy = df.copy()

In [148]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [149]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,...,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,...,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


In [92]:
df2.replace([], 'None', regex=True,inplace=True)

In [103]:
df2 = df2.mask(df2.applymap(type).eq(list) & ~df2.astype(bool))

In [104]:
df2

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-arax,NaN,"[infores:rtx-kg2, infores:semmeddb, [PMID:2976...","[infores:rtx-kg2, infores:dgidb, infores:arax]","[infores:rtx-kg2, infores:dgidb, infores:arax]","[infores:rtx-kg2, infores:semmeddb, [PMID:7696...","[infores:rtx-kg2, infores:semmeddb, [PMID:7696...",NaN,NaN,NaN,NaN,...,"[infores:rtx-kg2, infores:semmeddb, [PMID:2747...","[infores:rtx-kg2, infores:goa, [PMID:23871208]...",NaN,NaN,"[infores:rtx-kg2, infores:semmeddb, [PMID:1100...","[infores:rtx-kg2, infores:semmeddb, [PMID:1701...","[infores:rtx-kg2, infores:semmeddb, [PMID:1040...",NaN,NaN,NaN
ara-aragorn,NaN,"[infores:lincs, infores:spoke, -6.963625168134...","[72, [], infores:aragorn-ranker-ara, 1]","[1, [], infores:aragorn-ranker-ara, 1]",NaN,NaN,NaN,NaN,NaN,NaN,...,"[4, [], infores:aragorn-ranker-ara, 1]",NaN,"[537, [], infores:aragorn-ranker-ara, 1]",NaN,"[261, [], infores:aragorn-ranker-ara, 1]","[16, [], infores:aragorn-ranker-ara, 1]","[21, [], infores:aragorn-ranker-ara, 1]",NaN,NaN,"[[IntAct_1_norm_edges.jsonl], MI:0399, infores..."
ara-bte,NaN,"[[infores:semmeddb], [SEMMED Gene API], [PMID:...","[infores:translator-biothings-explorer, [edges...","[infores:translator-biothings-explorer, -, Fal...",NaN,"[infores:translator-biothings-explorer, pIC50,...","[infores:translator-biothings-explorer, [Cord1...","[[], [Clinical Risk KP API], [], 0.914, Logist...",NaN,NaN,...,"[infores:translator-biothings-explorer, False,...",NaN,NaN,NaN,NaN,NaN,"[[infores:semmeddb], [SEMMED Chemical API], [P...",NaN,NaN,NaN
ara-unsecret,NaN,NaN,NaN,"[95.05, PMID:29195078, https://clue.io/cmap, 1...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[95.64, PMID:29195078, https://clue.io/cmap, 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[False, ""(\""\""identifiers_org_registry:reactom..."
kp-genetics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kp-molecular,NaN,NaN,NaN,"[95.05, PMID:29195078, https://clue.io/cmap, 1...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[95.64, PMID:29195078, https://clue.io/cmap, 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ara-explanatory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ara-improving,"[infores:lincs, infores:spoke, infores:improvi...","[infores:lincs, infores:spoke, infores:improvi...","[pmid:25184630, infores:civic, infores:spoke, ...","[infores:lincs, infores:spoke, infores:improvi...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[infores:chembl, infores:spoke, infores:improv...","[infores:diseases, infores:spoke, infores:impr...","[infores:drugcentral, infores:spoke, infores:i...","[infores:bindingdb, infores:spoke, infores:imp...","[infores:bindingdb, infores:spoke, infores:imp...","[infores:string, infores:spoke, infores:improv..."
kp-cam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"[[infores:cam-kp], [infores:go-cam]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kp-textmining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

## Converting the Pk's to hyperlink

<br>

In [151]:
df.loc['pk'] = df.loc['pk'].apply(lambda x: make_hyperlink(x))

In [106]:
#df.rename({'pk_id': 'pk'}, inplace=True)

In [152]:
df.sort_index(inplace=True)

In [153]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,...,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,...,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [154]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [155]:
styled = df.style.applymap(highlight)

In [156]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 422,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 

In [117]:
import xlsxwriter

In [119]:
import xlwt

In [157]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [158]:
wks_name

'Workflow Progress Tracker_2021_09_07-12_29_08_AM'

In [114]:
#wks_name = 'Workflow Progress Tracker_2021_09_03-01_13_17_AM'

In [115]:
#wks_name = 'Workflow Progress Tracker_2021_08_26-03_56_45_AM'

In [122]:
writer = pd.ExcelWriter('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx', engine='xlsxwriter')

In [159]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [160]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 422,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 

In [161]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 598,...,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Error: 400,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,...,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Error: 400,Error: 400,Error: 598
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200,Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,Error: 504,No Results: 200
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

In [162]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [163]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [164]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_07-12_29_08_AM' id:1408731564>

In [133]:
wks2 = 'edge_attribute_source_' + date

In [134]:
wks2

'edge_attribute_source_2021_09_07-12_19_25_AM'

In [135]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_07-12_19_25_AM' id:970823174>

In [165]:
spreadsheet_key

'1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'

In [166]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [167]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [168]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
sh = wksh.worksheet(wks_name)

In [169]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_07-12_29_08_AM' id:1408731564>

In [170]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [171]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [172]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [173]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bo

In [174]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1408731564,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
    

In [175]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_07-12_29_08_AM' id:1408731564>

In [176]:
# Select a range
cell_list = sh.range('B17:Y19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_07-12_29_08_AM'!B17:Y19",
 'updatedRows': 3,
 'updatedColumns': 24,
 'updatedCells': 72}